In [1]:
import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 


In [2]:
#Initialize the client's ID and secret
cid ="89f3dc51e32147269a3c2e9a99bd7f87" 
secret = "647438b4afa04cb6822b5e3cb50d315a"


In [3]:
#Setting up the authorization with the API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
#Use the Playlist API to get the tracks from the playlists
def get_playlist_tracks(user_id, playlist_id):
    '''
    Get the contents in the playlist that includes the track name, track ID, and the artist's name and ID
    '''
    
    tracks = sp.user_playlist_tracks(user = user_id, playlist_id = playlist_id, fields='items.track.name,items.track.id,items.track.artists.name,items.track.artists.id,next')
    songs = tracks['items']

    while tracks['next']:
        tracks = sp.next(tracks)
        songs.extend(tracks['items'])

    return songs

In [5]:
#Get the top tracks from 2000-2019
top_2000 = get_playlist_tracks('spotify', '37i9dQZF1DWUZv12GM5cFk')

top_2001 = get_playlist_tracks('spotify', '37i9dQZF1DX9Ol4tZWPH6V')

top_2002 = get_playlist_tracks('spotify', '37i9dQZF1DX0P7PzzKwEKl')

top_2003 = get_playlist_tracks('spotify', '37i9dQZF1DXaW8fzPh9b08')

top_2004 = get_playlist_tracks('spotify', '37i9dQZF1DWTWdbR13PQYH')

top_2005 = get_playlist_tracks('spotify', '37i9dQZF1DWWzQTBs5BHX9')

top_2006 = get_playlist_tracks('spotify', '37i9dQZF1DX1vSJnMeoy3V')

top_2007 = get_playlist_tracks('spotify', '37i9dQZF1DX3j9EYdzv2N9')

top_2008 = get_playlist_tracks('spotify', '37i9dQZF1DWYuGZUE4XQXm')

top_2009 = get_playlist_tracks('spotify', '37i9dQZF1DX4UkKv8ED8jp')

top_2010 = get_playlist_tracks('spotify', '37i9dQZF1DXc6IFF23C9jj')

top_2011 = get_playlist_tracks('spotify', '37i9dQZF1DXcagnSNtrGuJ')

top_2012 = get_playlist_tracks('spotify', '37i9dQZF1DX0yEZaMOXna3')

top_2013 = get_playlist_tracks('spotify', '37i9dQZF1DX3Sp0P28SIer')

top_2014 = get_playlist_tracks('spotify', '37i9dQZF1DX0h0QnLkMBl4')

top_2015 = get_playlist_tracks('spotify', '37i9dQZF1DX9ukdrXQLJGZ')

top_2016 = get_playlist_tracks('spotify', '37i9dQZF1DX8XZ6AUo9R4R')

top_2017 = get_playlist_tracks('spotify', '37i9dQZF1DWTE7dVUebpUW')

top_2018 = get_playlist_tracks('spotify', '37i9dQZF1DXe2bobNYDtW8')

top_2019 = get_playlist_tracks('spotify', '37i9dQZF1DWVRSukIED0e9')

tops = [top_2000, top_2001, top_2002, top_2003, top_2004, top_2005, top_2006, top_2007, top_2008, top_2009, top_2010, top_2011, top_2012, top_2013, top_2014, top_2015, top_2016, top_2017, top_2018, top_2019]



In [6]:
#Label the year for the song
def add_year(songs, year):
    for song in songs:
        song['year'] = year

year = 2000
for top_year in tops:
    add_year(top_year, year)
    year += 1
    if year > 2019:
        break

In [7]:
top_2019[0]['track']['artists']

[{'id': '7jVv8c5Fj3E9VhNjxT4snq', 'name': 'Lil Nas X'},
 {'id': '60rpJ9SgigSd16DOAG7GSa', 'name': 'Billy Ray Cyrus'}]

In [8]:
#Create dataframe of all the tracks

full_song_ids = []
full_song_names = []
full_artists = []
full_artists_ids = []
song_years = []

for year in tops:
    for tracks in year:
        
        # list of song ids 
        full_song_ids.append(tracks['track']['id'])

        # list of song names  
        full_song_names.append(tracks['track']['name'])

        # list of artists and artists ID
        artists = []
        artists_id = []
        for artist in tracks['track']['artists']:
            artists.append(artist['name'])
            artists_id.append(artist['id'])
        full_artists.append(artists)
        full_artists_ids.append(artists_id)
        
        # list of years
        song_years.append(tracks['year'])

songs_df = pd.DataFrame(data = list(zip(full_song_ids, full_song_names, full_artists, full_artists_ids, song_years)), columns = ['songs_id', 'songs_name', 'artist_name', 'artist_id', 'year'])
display(songs_df.head())

,songs_id,songs_name,artist_name,artist_id,year
0,6naxalmIoLFWR0siv8dnQQ,Oops!...I Did It Again,[Britney Spears],[26dSoYclwsYLMAKD3tpOr4],2000
1,2m1hi0nfMR9vdGC8UcrnwU,All The Small Things,[blink-182],[6FBDaR13swtiWwGhX1WQsP],2000
2,3y4LxiYMgDl4RethdzpmNe,Breathe,[Faith Hill],[25NQNriVT2YbSW80ILRWJa],2000
3,0v1XpBHnsbkCn7iJ9Ucr1l,It's My Life,[Bon Jovi],[58lV9VcRSjABbAbfWS6skp],2000
4,4r8lRYnoOGdEi6YyI5OC1o,Bye Bye Bye,[*NSYNC],[6Ff53KvcvAj5U7Z1vojB5o],2000


In [9]:
#Use the Tracks API to get the audio features for all the tracks
def audio_feats(songs_df):
    features = sp.audio_features(songs_df['songs_id'])[0]
    songs_df['danceability'] = features['danceability']
    songs_df['energy'] = features['energy']
    songs_df['key'] = features['key']
    songs_df['loudness'] = features['loudness']
    songs_df['mode'] = features['mode']
    songs_df['speechiness'] = features['speechiness']
    songs_df['acousticness'] = features['acousticness']
    songs_df['instrumentalness'] = features['instrumentalness']
    songs_df['liveness'] = features['liveness']
    songs_df['valence'] = features['valence']
    songs_df['tempo'] = features['tempo']
    songs_df['duration'] = features['duration_ms']
    songs_df['time_signature'] = features['time_signature']
    return songs_df

In [10]:
#Add the track's audio features for all songs
songs_df = songs_df.apply(audio_feats, axis=1)

songs_df.head()

,songs_id,songs_name,artist_name,artist_id,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature
0,6naxalmIoLFWR0siv8dnQQ,Oops!...I Did It Again,[Britney Spears],[26dSoYclwsYLMAKD3tpOr4],2000,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,211160,4
1,2m1hi0nfMR9vdGC8UcrnwU,All The Small Things,[blink-182],[6FBDaR13swtiWwGhX1WQsP],2000,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,167067,4
2,3y4LxiYMgDl4RethdzpmNe,Breathe,[Faith Hill],[25NQNriVT2YbSW80ILRWJa],2000,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,250547,4
3,0v1XpBHnsbkCn7iJ9Ucr1l,It's My Life,[Bon Jovi],[58lV9VcRSjABbAbfWS6skp],2000,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,224493,4
4,4r8lRYnoOGdEi6YyI5OC1o,Bye Bye Bye,[*NSYNC],[6Ff53KvcvAj5U7Z1vojB5o],2000,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,200560,4


In [14]:
#Use the Artists API to get each artist's musical genres
def get_genres(artist_ids):
    genres = sp.artists(artist_ids)['artists'][0]['genres']
    return genres
artist_genres = songs_df['artist_id'].apply(get_genres)
artist_genres

0                         [dance pop, pop, post-teen pop]
1                        [pop punk, punk, socal pop punk]
2       [contemporary country, country, country dawn, ...
3                           [glam metal, hard rock, rock]
4               [boy band, dance pop, pop, post-teen pop]
                              ...                        
2024                                    [electropop, pop]
2025                     [grime, uk hip hop, ukg revival]
2026                    [melodic rap, pop rap, rap, trap]
2027    [afro dancehall, afroswing, uk dancehall, uk h...
2028                          [dfw rap, melodic rap, rap]
Name: artist_id, Length: 2029, dtype: object

In [15]:
songs_df['artist_genre'] = artist_genres
songs_df.head()

,songs_id,songs_name,artist_name,artist_id,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature,artist_genre
0,6naxalmIoLFWR0siv8dnQQ,Oops!...I Did It Again,[Britney Spears],[26dSoYclwsYLMAKD3tpOr4],2000,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,211160,4,"[dance pop, pop, post-teen pop]"
1,2m1hi0nfMR9vdGC8UcrnwU,All The Small Things,[blink-182],[6FBDaR13swtiWwGhX1WQsP],2000,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,167067,4,"[pop punk, punk, socal pop punk]"
2,3y4LxiYMgDl4RethdzpmNe,Breathe,[Faith Hill],[25NQNriVT2YbSW80ILRWJa],2000,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,250547,4,"[contemporary country, country, country dawn, ..."
3,0v1XpBHnsbkCn7iJ9Ucr1l,It's My Life,[Bon Jovi],[58lV9VcRSjABbAbfWS6skp],2000,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,224493,4,"[glam metal, hard rock, rock]"
4,4r8lRYnoOGdEi6YyI5OC1o,Bye Bye Bye,[*NSYNC],[6Ff53KvcvAj5U7Z1vojB5o],2000,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,200560,4,"[boy band, dance pop, pop, post-teen pop]"


In [ ]:
#Output DF to a CSV
#songs_df.to_csv('top_songs.csv')

#Read the CSV
#pd.read_csv('top_songs.csv')